In [34]:
import cv2
import os
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

def preprocess_image(img):
    # Converts to greyscale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return gray_img

def extract_features(img):
    # texture extraction
    textures = cv2.HuMoments(cv2.moments(cv2.equalizeHist(img))).flatten()
    return textures

def resize_images(images, size=(128, 128)):
    resized_images = [cv2.resize(img, size) for img in images]
    return resized_images

def data_augmentation(images, num_rotations=4, flip=True):
    augmented_images = []
    for img in images:
        augmented_images.append(img)
        for _ in range(num_rotations):
            rotated_img = np.rot90(img, k=np.random.randint(1, 4))
            augmented_images.append(rotated_img)
            if flip:
                flipped_img = cv2.flip(rotated_img, np.random.randint(-1, 2))
                augmented_images.append(flipped_img)
    return augmented_images

# Healthy plant path
healthy_images_dir = "/kaggle/input/cotton-dataset/Dataset/healthy"

# Sick plant path
rotten_images_dir = "/kaggle/input/cotton-dataset/Dataset/curl_stage2+sooty"

# healthy image pre-processing
healthy_images = [cv2.imread(os.path.join(healthy_images_dir, filename), cv2.IMREAD_COLOR) for filename in os.listdir(healthy_images_dir)]
healthy_images = [preprocess_image(img) for img in healthy_images]
healthy_images = resize_images(healthy_images)
healthy_labels = ['Healthy'] * len(healthy_images)

# rotten images pre-processing
rotten_images = [cv2.imread(os.path.join(rotten_images_dir, filename), cv2.IMREAD_COLOR) for filename in os.listdir(rotten_images_dir)]
rotten_images = [preprocess_image(img) for img in rotten_images]
rotten_images = resize_images(rotten_images)
rotten_labels = ['Rotten'] * len(rotten_images)


augmented_healthy_images = data_augmentation(healthy_images)
augmented_rotten_images = data_augmentation(rotten_images)

# Combine healthy and rotten images
all_images = augmented_healthy_images + augmented_rotten_images
all_labels = ['Healthy'] * len(augmented_healthy_images) + ['Rotten'] * len(augmented_rotten_images)

# lists->np arrays
all_images = np.array(all_images)
all_labels = np.array(all_labels)

# feature extractions
all_features = np.array([extract_features(img) for img in all_images])

# Standardize features
scaler = StandardScaler()
all_features_scaled = scaler.fit_transform(all_features)

# RF training
clf = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=5, min_samples_leaf=2, random_state=42)
clf.fit(all_features_scaled, all_labels)

#image classification
def classify_images(image_folder):
    predictions = []
    for filename in os.listdir(image_folder):
        # read image,pre-process
        image = cv2.imread(os.path.join(image_folder, filename))
        img_preprocessed = preprocess_image(image)
        
        
        #image rezing, feature extraction, and scale
        resized_img = cv2.resize(img_preprocessed, (128, 128))
        img_features = extract_features(resized_img)
        scaled_features = scaler.transform([img_features])
        
        # class prediction
        prediction = clf.predict(scaled_features)[0]
        predictions.append((filename, prediction))
    return predictions

# Group the images by category
dataset_dir = "/kaggle/input/cotton-dataset/Dataset"
predicted_labels = []
true_labels = []
for category in os.listdir(dataset_dir):
    category_dir = os.path.join(dataset_dir, category)
    if os.path.isdir(category_dir):
        print(f"\n{category} Images Classification:")
        category_predictions = classify_images(category_dir)
        for filename, prediction in category_predictions:
            print(f"{filename}: {prediction}")
            predicted_labels.append(prediction)
            if category == 'healthy':
                true_labels.append('Healthy')
            else:
                true_labels.append('Rotten')

#f1 score of all png
f1 = f1_score(true_labels, predicted_labels, average='macro', zero_division=0)
print(f"\nOverall F1 Score: {f1}")




curl_stage1+sooty Images Classification:
CS1_Sooty (27).png: Rotten
CS1_Sooty (33).png: Healthy
CS1_Sooty (4).png: Rotten
CS1_Sooty (30).png: Rotten
CS1_Sooty (19).png: Healthy
CS1_Sooty (12).png: Healthy
CS1_Sooty (1).jpg: Healthy
CS1_Sooty (24).png: Rotten
CS1_Sooty (4).jpg: Rotten
CS1_Sooty (18).png: Rotten
CS1_Sooty (8).png: Healthy
CS1_Sooty (44).png: Rotten
CS1_Sooty (13).png: Rotten
CS1_Sooty (34).png: Healthy
CS1_Sooty (23).png: Rotten
CS1_Sooty (11).png: Rotten
CS1_Sooty (32).png: Rotten
CS1_Sooty (3).png: Rotten
CS1_Sooty (43).png: Rotten
CS1_Sooty (16).png: Rotten
CS1_Sooty (7).png: Rotten
CS1_Sooty (3).jpg: Healthy
CS1_Sooty (40).png: Rotten
CS1_Sooty (9).png: Healthy
CS1_Sooty (35).png: Rotten
CS1_Sooty (47).png: Rotten
CS1_Sooty (2).jpg: Healthy
CS1_Sooty (21).png: Healthy
CS1_Sooty (10).png: Rotten
CS1_Sooty (20).png: Rotten
CS1_Sooty (46).png: Rotten
CS1_Sooty (45).png: Rotten
CS1_Sooty (29).png: Rotten
CS1_Sooty (36).png: Rotten
CS1_Sooty (39).png: Rotten
CS1_Sooty (4